In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.chains import RetrievalQA

# Initialize LLM
llm = ChatOpenAI()

# Define the cache directory
cache_dir = LocalFileStore("./.cache")

# Initialize the text splitter
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=300,
    chunk_overlap=30,
)

# Load and split the document
loader = UnstructuredFileLoader("./files/BAU_Report_May.docx")
docs = loader.load_and_split(text_splitter=splitter)

# Initialize OpenAI embeddings
embeddings = OpenAIEmbeddings()

# Initialize cache-backed embeddings
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(
    embeddings, cache_dir
)

# Initialize the vector store using the cached embeddings
vectorstore = FAISS.from_documents(docs, cached_embeddings)

# Initialize the retrieval chain
chain = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="map_rerank",
    retriever=vectorstore.as_retriever(),
)

# Run the chain with a query
chain.run("Describe 1 acheivement")




ImportError: cannot import name 'SimpleOutputParser' from 'langchain.output_parsers' (c:\Users\Jonathan\Documents\fullstack-gpt\env\Lib\site-packages\langchain\output_parsers\__init__.py)